# CITI BIKE

This notebook combines data from January 2023 and January 2024 from Citi Bike (https://citibikenyc.com/system-data).

It concatenates the three data files obtained from citibike and calculates trip durations and distance for each bike trip in the concatenated dataframe, as well as the day of the week. 

The resulting df is saved and used to build a Tableau workbook that can be found here:https://public.tableau.com/app/profile/catarina.ferreira2233/viz/Citi_Bike_17084488357830/Story2

In [1]:
# import dependencies
import pandas as pd
import datetime as dt
import calendar
import geopy.distance

## Import data and concatenate dataframes

In [2]:
# read in data
jan_23 = pd.read_csv('data/202301-citibike-tripdata.csv')
jan_24_1 = pd.read_csv('data/202401-citibike-tripdata_1.csv')
jan_24_2 = pd.read_csv('data/202401-citibike-tripdata_2.csv')

C:\Users\catis\AppData\Local\Temp\ipykernel_21272\2957987616.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  jan_23 = pd.read_csv('data/202301-citibike-tripdata.csv')
C:\Users\catis\AppData\Local\Temp\ipykernel_21272\2957987616.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  jan_24_1 = pd.read_csv('data/202401-citibike-tripdata_1.csv')
C:\Users\catis\AppData\Local\Temp\ipykernel_21272\2957987616.py:4: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  jan_24_2 = pd.read_csv('data/202401-citibike-tripdata_2.csv')


In [3]:
# put all dfs together
merged_df = pd.concat([jan_23, jan_24_1, jan_24_2], ignore_index=True)

# check it all looks ok
merged_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,4A86C1475DCCADA0,classic_bike,2023-01-26 10:53:44,2023-01-26 11:05:17,E 53 St & Madison Ave,6659.03,Columbus Ave & W 72 St,7175.05,40.759724,-73.973664,40.777057,-73.978985,member
1,AE02E1FF7E264874,classic_bike,2023-01-04 11:51:54,2023-01-04 12:07:09,Halsey St & Ralph Ave,4334.02,Clinton Ave & Myrtle Ave,4651.02,40.684970,-73.922755,40.693261,-73.968896,casual
2,D3F9A2A71AD244C6,classic_bike,2023-01-04 17:26:48,2023-01-04 17:43:29,Bank St & Hudson St,5922.08,W 52 St & 11 Ave,6926.01,40.736566,-74.006092,40.767272,-73.993929,casual
3,3D48C6F33AFEF329,classic_bike,2023-01-04 08:31:11,2023-01-04 08:42:12,E 53 St & Madison Ave,6659.03,W 45 St & 8 Ave,6676.02,40.759711,-73.974023,40.759291,-73.988597,member
4,4865926C3D97BA01,electric_bike,2023-01-03 22:15:34,2023-01-03 22:30:35,Frederick Douglass Blvd & W 112 St,7631.23,Central Park S & 6 Ave,6876.04,40.801694,-73.957145,40.765909,-73.976342,casual


In [4]:
merged_df.dropna(inplace=True)

## Calculate trip duration and day of the week
Source code: https://pieriantraining.com/python-tutorial-how-to-compare-dates-in-python/

In [5]:
# check to see if there's a need to change data types
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3798501 entries, 0 to 3809576
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 405.7+ MB


In [6]:
# first change started and ended at to datetime
merged_df['started_at'] = merged_df['started_at'].astype('datetime64[ns]')
merged_df['ended_at'] = merged_df['ended_at'].astype('datetime64[ns]')

# calculate ride duration and add col to df
merged_df["duration"] = (merged_df["ended_at"] - merged_df["started_at"]).dt.total_seconds() / 60.0

In [7]:
# calculate day of the week
## weekday returns numbers 0-6 where 0 = Monday, 1 = Tuesday and so on until 6 = Sunday
merged_df["weekday"] = merged_df["started_at"].dt.weekday 

## Calculate trip distance
Source code: https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude

In [8]:
distance = []

for index, row in merged_df.iterrows():
    coords_1 = (row["start_lat"],row["start_lng"])
    coords_2 = (row["end_lat"],row["end_lng"])
    distance.append(geopy.distance.geodesic(coords_1, coords_2).km)

In [12]:
merged_df["distance"] = distance

## Save data

In [13]:
# just check everything looks ok
merged_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration,weekday,distance
0,4A86C1475DCCADA0,classic_bike,2023-01-26 10:53:44,2023-01-26 11:05:17,E 53 St & Madison Ave,6659.03,Columbus Ave & W 72 St,7175.05,40.759724,-73.973664,40.777057,-73.978985,member,11.550000,3,1.976582
1,AE02E1FF7E264874,classic_bike,2023-01-04 11:51:54,2023-01-04 12:07:09,Halsey St & Ralph Ave,4334.02,Clinton Ave & Myrtle Ave,4651.02,40.684970,-73.922755,40.693261,-73.968896,casual,15.250000,2,4.007481
2,D3F9A2A71AD244C6,classic_bike,2023-01-04 17:26:48,2023-01-04 17:43:29,Bank St & Hudson St,5922.08,W 52 St & 11 Ave,6926.01,40.736566,-74.006092,40.767272,-73.993929,casual,16.683333,2,3.561285
3,3D48C6F33AFEF329,classic_bike,2023-01-04 08:31:11,2023-01-04 08:42:12,E 53 St & Madison Ave,6659.03,W 45 St & 8 Ave,6676.02,40.759711,-73.974023,40.759291,-73.988597,member,11.016667,2,1.231463
4,4865926C3D97BA01,electric_bike,2023-01-03 22:15:34,2023-01-03 22:30:35,Frederick Douglass Blvd & W 112 St,7631.23,Central Park S & 6 Ave,6876.04,40.801694,-73.957145,40.765909,-73.976342,casual,15.016667,1,4.291535


In [14]:
# save data
merged_df.to_csv('data/merged_data.csv')